<a href="https://colab.research.google.com/github/Jsgithubchannel/Jsgithubchannel/blob/main/situation_model_changed_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 코랩 환경 설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [2]:
#Kobert 파일 로드 from github
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-1o15z8cf
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-1o15z8cf


In [3]:
# 환경 설정 (필요한 모듈 import)
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
#GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# 2. 데이터셋 전처리
import pandas as pd
chat_data = pd.read_excel('/content/drive/MyDrive/capstone_design/reduced_situation_dataset.xlsx')

In [10]:
print(len(chat_data)) # 데이터 개수
chat_data.sample(n=10) # 랜덤 데이터 10개 출력

49660


,situation,sentence1,sentence2,sentence3
42753,"진로, 취업, 직장",친구가 이번에 대기업에 취직했어.,나보다 공부도 못하던 친구인데 너무 질투가 나.,나도 준비했던 곳인데 나는 떨어졌거든.
34993,"재정, 은퇴, 노후준비",요즘 경제적으로 집안 살림이 어려워서 너무 힘들다.,아니야. 말을 들어주는것만으로 고마워 사실 너무 괴로운데 털어놓을 때가 없었거든.,그래. 애기하다보니 이제 괴로운 감정도 어느 정도 사라진 것 같아.
43864,학교폭력/따돌림,나 오늘 학교 친구들이 따돌림을 모의하는 것을 들어버렸어. 내가 너무 한심하고 우울해.,나는 그걸 듣고도 무서워서 아무 것도 하지 못했어.,학교 선생님에게 익명으로 편지를 쓸 거야. 가장 안전하게 친구를 도와줄 수 있을 것...
31712,재정,노인정에서 불우이웃돕기 모금을 했는데 나만 약값 내기도 버거워서 돈을 안 냈어.,돈을 내자니 내가 쓸 돈이 없고 그렇다고 안 내자니 부끄럽고. 어떻게 하지?,그냥 돈을 안 내야 할 것 같아. 나도 너무 힘들거든.
41410,"진로, 취업, 직장",그 남자와 회사를 나와서 같이 사업을 하기로 했어.,축하하긴 일러 퇴사 할 때까진 비밀로 하려고 했는데 소문이 나서 다니기가 너무 힘들어.,사람 구해질 때까진 다니긴 하는데 너무 고역이다. 도축장에 끌려가는 기분이야.
1742,가족관계,엄마는 나보고 요즘 왜 이렇게 축 쳐져 있냐고 물어봐.,딱히 어떤 사건이 있었던 건 아니야. 학년이 올라가고 학업 스트레스 인 것 같기도 ...,별 일이 있는 건 아닌데 걱정 하실까봐 쉽게 얘기가 안 되네.
40019,"진로, 취업, 직장",내가 몇 날 며칠을 매달리며 열심히 준비했던 일정이 취소돼서 너무 허무해.,내가 기울인 노력은 다 물거품이 되는 걸까. 아무런 의욕도 나지 않아.,이번 기회로 좋은 경험했다고 생각하는 것도 나쁘지 않을 것 같아.
6591,"건강, 죽음",집에 에이아이만이 날 반겨주네.,있어도 연락두절된지 한참됐어. 이렇게 외롭게 죽겠지 뭐.,짐이 되니까 떠난거지. 허공에 대고 소리치는 기분이 들어.
32250,"재정, 은퇴, 노후준비",처음에는 일을 할 때마다 은퇴가 두려웠지만 지금은 되레 은퇴할 수 있다는 말이 신이 나.,이제는 은퇴 후 노후준비를 이제 할 수 있어서 기뻐.,집에 가서 노후준비에 관한 책을 읽어봐야겠어.
6187,건강,건강 때문에 운동을 해야하는데 하기 싫어서 스트레스 받아.,즐겁게 운동하면서 스트레스 받고 싶지 않아.,혼자 해서 재미가 없었던 것 같아. 사람들과 함께 할 수 있는 운동을 찾아봐야겠어.


In [11]:
situations = ['가족관계','학업 및 진로','학교폭력/따돌림','대인관계(부부, 자녀)','대인관계','연애, 결혼, 출산', '진로, 취업, 직장', '재정, 은퇴, 노후준비', '건강, 죽음', '건강', '직장, 업무 스트레스', '재정'] # 감정 대분류
for i in range(len(situations)):
  chat_data.loc[(chat_data['situation'] == situations[i]), 'situation'] = i

In [12]:
data_list = []
for label, sentence1, sentence2, sentence3 in zip(chat_data['situation'], chat_data['sentence1'], chat_data['sentence2'], chat_data['sentence3']):
    data = []
    sentence = sentence1+' '+sentence2+' '+sentence3
    data.append(sentence)
    data.append(str(label))

    data_list.append(data)

In [13]:
data_list[49000]

['공부가 내 길이 아니라 공부를 안하는 것 뿐이야. 공부가 내 길이 아니라 가수가 내 길이라서 안하는거 뿐이야. 난 노래 연습을 할거야 그거면 돼.',
 '1']

In [14]:
# 3. Train data & Test data
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [15]:
# 4. KoBERT 입력 데이터로 만들기

# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [47]:
# Setting parameters
max_len = 200
batch_size = 16
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
print(len(dataset_train))
print(len(dataset_test))

39728
9932


In [18]:
#토큰화
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

#BERT 모델, Vocabulary 불러오기
from kobert.pytorch_kobert import get_kobert_model
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file) 

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ntmd7haw/kobert-tokenizer_e8affdc754834c718d7facc9024ade40
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ntmd7haw/kobert-tokenizer_e8affdc754834c718d7facc9024ade40


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [19]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok,vocab,  max_len, True, False)

In [20]:
data_train[0]

(array([   2, 4162, 6356, 5468, 7234, 1846, 3111, 5398, 5859, 3135, 7828,
        4627, 5330, 1567, 3194, 5940, 6022, 2752, 5931, 3191, 3574, 6585,
        1852, 5760,  517, 5617, 7096, 3128,  517, 5499, 7350, 5663, 5782,
          54,   54, 2251, 7832, 3191, 1958, 6150, 2307, 5920, 6113, 6113,
        7784, 4966, 1562, 5330, 3105, 5850,  517, 5617, 6368, 3059, 6553,
        5100, 5671, 5702,   54, 3192, 5760, 4799, 7659, 6116,  517, 7375,
        5357, 6553, 1610, 2437, 7088,  775, 2251, 7788, 4435, 3135, 7811,
        4077, 6116, 2086, 6291, 5859, 1189, 4156, 7836, 5377, 6844,   54,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [21]:
# torch 형식의 dataset
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [22]:
# 5. KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=12, ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [48]:
# KoBERT 모델 학습시키기
for e in range(epoch, epoch + num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch = epoch + num_epochs

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/2483 [00:00<?, ?it/s]

epoch 11 batch id 1 loss 0.13229189813137054 train acc 1.0
epoch 11 batch id 201 loss 0.44337621331214905 train acc 0.8861940298507462
epoch 11 batch id 401 loss 0.32615992426872253 train acc 0.8884039900249376
epoch 11 batch id 601 loss 0.5481113791465759 train acc 0.8951747088186356
epoch 11 batch id 801 loss 0.8947399854660034 train acc 0.8987983770287141
epoch 11 batch id 1001 loss 0.2530180811882019 train acc 0.9024100899100899
epoch 11 batch id 1201 loss 0.4307253360748291 train acc 0.9038301415487094
epoch 11 batch id 1401 loss 0.120720274746418 train acc 0.904621698786581
epoch 11 batch id 1601 loss 0.5679891109466553 train acc 0.9051374141161774
epoch 11 batch id 1801 loss 0.1767856627702713 train acc 0.9059203220433093
epoch 11 batch id 2001 loss 0.06141643226146698 train acc 0.9059220389805097
epoch 11 batch id 2201 loss 0.39171096682548523 train acc 0.9063209904588824
epoch 11 batch id 2401 loss 0.17233364284038544 train acc 0.9060287380258226
epoch 11 train acc 0.905457108

  0%|          | 0/621 [00:00<?, ?it/s]

epoch 11 test acc 0.6761943102522813


  0%|          | 0/2483 [00:00<?, ?it/s]

epoch 12 batch id 1 loss 0.5188328623771667 train acc 0.6875
epoch 12 batch id 201 loss 0.4457877278327942 train acc 0.8989427860696517
epoch 12 batch id 401 loss 0.10552340000867844 train acc 0.9085099750623441
epoch 12 batch id 601 loss 0.02348652109503746 train acc 0.91358153078203
epoch 12 batch id 801 loss 0.2933671176433563 train acc 0.9131554307116105
epoch 12 batch id 1001 loss 0.23795518279075623 train acc 0.915521978021978
epoch 12 batch id 1201 loss 0.31224125623703003 train acc 0.9164238134887593
epoch 12 batch id 1401 loss 0.29332026839256287 train acc 0.9188526052819415
epoch 12 batch id 1601 loss 0.08711714297533035 train acc 0.9190740162398501
epoch 12 batch id 1801 loss 0.10214804857969284 train acc 0.9190033314825097
epoch 12 batch id 2001 loss 0.05497061833739281 train acc 0.9197588705647176
epoch 12 batch id 2201 loss 0.45225396752357483 train acc 0.9201783280327124
epoch 12 batch id 2401 loss 0.22408334910869598 train acc 0.9197209496043315
epoch 12 train acc 0.920

  0%|          | 0/621 [00:00<?, ?it/s]

epoch 12 test acc 0.6883722490606549


In [50]:
# 저장하기
state = {
        'epoch': num_epochs,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'loss': loss,
}
path ="/content/drive/MyDrive/capstone_design/model_epoch12.pth"
torch.save(state,path)

In [51]:
# 불러오기
checkpoint = torch.load("/content/drive/MyDrive/capstone_design/model_epoch12.pth")
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# model.train()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [58]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 1:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 2:
                test_eval.append("학교폭력/따돌림")
            elif np.argmax(logits) == 3:
                test_eval.append("대인관계(부부, 자녀)")
            elif np.argmax(logits) == 4:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 5:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 6:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 7:
                test_eval.append("재정, 은퇴, 노후준비")
            elif np.argmax(logits) == 8:
                test_eval.append("건강, 죽음")
            elif np.argmax(logits) == 9:
                test_eval.append("건강")
            elif np.argmax(logits) == 10:
                test_eval.append("직장, 업무 스트레스")
            elif np.argmax(logits) == 11:
                test_eval.append("재정")

            print(logits)
        print(">> 입력하신 내용에서 " + test_eval[0] + " 상황입니다.")

In [59]:
#질문 무한반복하기! 0 입력시 종료

end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 공부가 내 길이 아니라 공부를 안하는 것 뿐이야. 공부가 내 길이 아니라 가수가 내 길이라서 안하는거 뿐이야. 난 노래 연습을 할거야 그거면 돼.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[-0.08626291  7.5850387   1.0644163   0.12527494  0.2832121  -2.4605815
  0.5631924  -0.43216223 -1.5626361   0.08037304 -2.4776974  -0.572239  ]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 공부는 힘들어
[-0.8184731   7.1750045   1.5238721   0.7509221   0.39652944 -2.6210127
 -0.0325031   0.01507865 -1.4918983   0.36574253 -2.4716053  -0.8044045 ]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 돈버는 것도 힘들어
[-2.5421252  -0.7501472  -1.6436696  -0.95584273  0.85087025 -2.2679324
  1.2098463   2.2372916  -1.0911473  -0.9288866  -0.34687385  5.5733123 ]
>> 입력하신 내용에서 재정 상황입니다.


하고싶은 말을 입력해주세요 : 삶의 의미는 뭘까?
[-0.1750377   2.8037302   0.6891271  -1.555724    0.370161   -1.8484235
  1.3590399   0.45635992  2.256129   -1.1854153  -2.4654038  -0.20953509]
>> 입력하신 내용에서 학업 및 진로 상황입니다.


하고싶은 말을 입력해주세요 : 요즘 몸이 안좋아서 필라테스 다녀왔어
[ 1.230831   -1.0924592  -1.3921441  -2.224237    1.3496826  -0.09393229
 -2.1893322  -1.9079479   6.3059173   1.1123224  -2.4035513  -0.3818108 ]
>> 입력하신 내용에서 건강, 죽음 상황입니다.


하고싶은 말

In [ ]:
#캐시 삭제
import gc
gc.collect()
torch.cuda.empty_cache()